In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision
import os
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset, Dataset
from PIL import Image
!cp -r "/kaggle/input/rsna-models/facebookresearch_dinov2_main (1)/root/.cache/torch/hub/facebookresearch_dinov2_main" /kaggle/working/dinov2
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

cp: cannot stat '/kaggle/input/rsna-models/facebookresearch_dinov2_main (1)/root/.cache/torch/hub/facebookresearch_dinov2_main': No such file or directory


In [2]:
from transformers import AutoImageProcessor, AutoModel
processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/base/1/')
model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/base/1/')
model = model.cuda()

2025-11-01 00:19:48.317210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761956388.507721      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761956388.564515      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWar

In [3]:
embeds = []
targets = [[] for i in range(5)]
counter = 0
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
#transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224, 224)), transforms.Normalize(mean, std)])
train_df = pd.read_csv("/kaggle/input/csiro-biomass/train.csv")
root = "/kaggle/input/csiro-biomass/"
for i in range(len(train_df)):
    entry = train_df.iloc[i]
    file_path = root + entry['image_path']
    y = torch.tensor([[entry['target']]])
    targets[i % 5].append(y)
    if i % 5 == 0:
        img = Image.open(file_path)
        x = torch.tensor(processor(img).pixel_values)
        with torch.no_grad():
            x = x.cuda()
            embeds.append(model(x).pooler_output.cpu())
            counter += 1
            if counter % 100 == 0:
                print(f"{counter} batches processed.")

/tmp/ipykernel_19/3300202635.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  x = torch.tensor(processor(img).pixel_values)


100 batches processed.
200 batches processed.
300 batches processed.


In [4]:
import random
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

# Create indices and shuffle once
lst = list(range(len(embeds)))
random.seed(42)
random.shuffle(lst)

# Create multiple random 80/20 splits
n_splits = 5
splits = []

for i in range(n_splits):
    # Reshuffle for each split while maintaining same splits across targets
    temp_lst = lst.copy()
    random.seed(42 + i)  # Different seed for each split
    random.shuffle(temp_lst)
    
    split_point = int(len(temp_lst) * 0.8)
    train_idxs = temp_lst[:split_point]
    val_idxs = temp_lst[split_point:]
    splits.append((train_idxs, val_idxs))

# Convert embeds to numpy array once for efficiency
embeds_np = np.array(torch.cat(embeds))
regressors = [[None for i in range(5)] for j in range(5)]
# Now iterate through each target
for i in range(5):
    print(f"\n=== Target {i+1} ===")
    targets_np = np.array(torch.cat(targets[i]))
    
    split_scores = []
    
    for split_idx, (train_idxs, val_idxs) in enumerate(splits):
        print(f"Fold {split_idx+1}:")
        X_train, y_train = embeds_np[train_idxs], targets_np[train_idxs]
        X_val, y_val = embeds_np[val_idxs], targets_np[val_idxs]
        reg = Lasso()
        reg.fit(X_train, y_train)
        train_preds = reg.predict(X_train)
        train_preds[train_preds < 0.0] = 0.0
        train_r2 = r2_score(y_train, train_preds)
        val_preds = reg.predict(X_val)
        val_preds[val_preds < 0.0] = 0.0
        val_r2 = r2_score(y_val, val_preds)
        print(f"  Train R²: {train_r2:.4f}")
        print(f"  Val R²: {val_r2:.4f}")
        split_scores.append((train_r2, val_r2))
        regressors[i][split_idx] = reg
    
    # Print summary for this target
    avg_train_r2 = np.mean([score[0] for score in split_scores])
    avg_val_r2 = np.mean([score[1] for score in split_scores])
    print(f"\nTarget {i+1} Average:")
    print(f"  Avg Train R²: {avg_train_r2:.4f}")
    print(f"  Avg Val R²: {avg_val_r2:.4f}")


=== Target 1 ===
Fold 1:
  Train R²: 0.6520
  Val R²: 0.4083
Fold 2:
  Train R²: 0.6006
  Val R²: 0.6426
Fold 3:
  Train R²: 0.6318
  Val R²: 0.5823
Fold 4:
  Train R²: 0.6516
  Val R²: 0.4472
Fold 5:
  Train R²: 0.6311
  Val R²: 0.4461

Target 1 Average:
  Avg Train R²: 0.6334
  Avg Val R²: 0.5053

=== Target 2 ===
Fold 1:
  Train R²: 0.4422
  Val R²: 0.3771
Fold 2:
  Train R²: 0.4612
  Val R²: 0.1990
Fold 3:
  Train R²: 0.4716
  Val R²: 0.3107
Fold 4:
  Train R²: 0.4591
  Val R²: 0.3874
Fold 5:
  Train R²: 0.4195
  Val R²: 0.4719

Target 2 Average:
  Avg Train R²: 0.4507
  Avg Val R²: 0.3492

=== Target 3 ===
Fold 1:
  Train R²: 0.8262
  Val R²: 0.7079
Fold 2:
  Train R²: 0.8337
  Val R²: 0.6208
Fold 3:
  Train R²: 0.8339
  Val R²: 0.6039
Fold 4:
  Train R²: 0.8342
  Val R²: 0.6463
Fold 5:
  Train R²: 0.8227
  Val R²: 0.7007

Target 3 Average:
  Avg Train R²: 0.8301
  Avg Val R²: 0.6559

=== Target 4 ===
Fold 1:
  Train R²: 0.7606
  Val R²: 0.5641
Fold 2:
  Train R²: 0.7942
  Val R²

In [5]:
mapping = {"Dry_Clover_g": 0, "Dry_Dead_g": 1, "Dry_Green_g": 2, "Dry_Total_g": 3, "GDM_g": 4}

In [6]:
test_embeds = {}
counter = 0
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224, 224)), transforms.Normalize(mean, std)])
test_df = pd.read_csv("/kaggle/input/csiro-biomass/test.csv")
root = "/kaggle/input/csiro-biomass/"
sample_ids = []
for i in range(len(test_df)):
    entry = test_df.iloc[i]
    file_path = root + entry['image_path']
    sample_id = entry['sample_id']
    #y = torch.tensor([[entry['target']]])
    if sample_id not in sample_ids:
        img = Image.open(file_path)
        x = torch.tensor(processor(img).pixel_values)
        with torch.no_grad():
            x = x.cuda()
            test_embeds[sample_id.split("_")[0]] = model(x).pooler_output.cpu()
            counter += 1
        sample_ids.append(sample_id)
    if counter % 100 == 0:
        print(f"{counter} batches processed.")

In [7]:
predictions = []
sample_ids = []
test_df = pd.read_csv("/kaggle/input/csiro-biomass/test.csv")
for i in range(len(test_df)):
    try:
        entry = test_df.iloc[i]
        X = np.array(test_embeds[entry['sample_id'].split("__")[0]])
        sample_ids.append(entry['sample_id'])
        models = regressors[mapping[entry['sample_id'].split("__")[1]]]
        prediction = 0.0
        for item in models:
            single_pred = item.predict(X)
            if single_pred < 0.0:
                single_pred = 0.0
            prediction += single_pred
        prediction = prediction / 5
        predictions.append(float(prediction))
    except Exception as e:
        predictions.append(0.0)

/tmp/ipykernel_19/131700968.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions.append(float(prediction))
/tmp/ipykernel_19/131700968.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions.append(float(prediction))
/tmp/ipykernel_19/131700968.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions.append(float(prediction))
/tmp/ipykernel_19/131700968.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is depre

In [8]:
%cd /kaggle/working
submission = pd.DataFrame({
    'sample_id': sample_ids,
    'target': predictions
})

submission.to_csv('submission.csv', index=False)
submission

/kaggle/working


,sample_id,target
0,ID1001187975__Dry_Clover_g,1.975965
1,ID1001187975__Dry_Dead_g,22.272205
2,ID1001187975__Dry_Green_g,31.084332
3,ID1001187975__Dry_Total_g,55.230492
4,ID1001187975__GDM_g,28.500387
